In [345]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import IPython
from tqdm import tqdm

In [346]:
# Mac OS GPU acceleration or CPU
device = torch.device("cpu" if torch.backends.mps.is_available() else "cpu")
print(device)

cpu


In [347]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [348]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [349]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [350]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [351]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [352]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [353]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
print(h.shape, W2.shape)
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability bc we exponentiate in next step
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()

loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

torch.Size([32, 64]) torch.Size([64, 27])


tensor(3.3196, grad_fn=<NegBackward0>)

In [354]:
bngain.shape, bnraw.shape, hpreact.shape, bnbias.shape

(torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [355]:
# Manually compute all the gradients in the backward pass
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

dprobs = 1/probs * dlogprobs

dcounts_sum_inv = counts * dprobs
dcounts_sum_inv = dcounts_sum_inv[range(n), Yb].unsqueeze(-1)

dcounts_sum = - counts_sum**-2 * dcounts_sum_inv

dcounts = counts_sum_inv * dprobs # First branch of dcounts must add to the second one to produce the true dcounts value. Fascinating.

dcounts += torch.ones_like(counts) * dcounts_sum # Addition is a "router" of derivatives. It just flows through and is replicated.

dnorm_logits = norm_logits.exp() * dcounts

dlogit_maxes = torch.sum(-dnorm_logits, dim=-1, keepdim=True) # the downsample of shape leads us to sum the gradients that are involved with each batch example

dlogits = dnorm_logits.clone()

dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T # just match the shapes, there's math behind it but this is easier.
dW2 = h.T @ dlogits # just match the shapes, there's math behind it but this is easier.
db2 = dlogits.sum(0) # just match the shapes, there's math behind it but this is easier.

dhpreact = (1 - h**2) * dh
dbnraw = bngain * dhpreact
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnbias = dhpreact.sum(0, keepdim=True)

dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

dbnvar = (-0.5 * (bnvar + 1e-5) **-1.5) * dbnvar_inv

dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar

dbndiff += (2*bndiff) * dbndiff2

dhprebn = dbndiff.clone()
dbnmeani = (- torch.ones_like(bndiff) * dbndiff).sum(0)

dhprebn += 1.0 / n * (torch.ones_like(hprebn) * dbnmeani)

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0, keepdim=True)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]

In [356]:
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bnraw', dbnraw, bnraw)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bndiff', dbndiff, bndiff)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('hprebn', dhprebn, hprebn)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

In [357]:
# Fast version of the cross entropy loss without going into each of the individual components as the mathematical expression simplifies significantly.
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff: ', (loss_fast - loss).item())

# I didn't do the derivation here but it can be done.

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits)


3.319556713104248 diff:  -2.384185791015625e-07
logits          | exact: False | approximate: True  | maxdiff: 4.889443516731262e-09


In [358]:
# Fast version of the batch norm layer without going into individual components
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True))
print('max diff:', (hpreact_fast - hpreact).abs().max())

# I didn't do the derivation here but it can be done.

dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)* bnraw * (dhpreact * bnraw).sum(0))
cmp('hprebn', dhprebn, hprebn)

max diff: tensor(2.7633, grad_fn=<MaxBackward1>)
hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [359]:
# Put it all together using a manual backward pass

batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
with torch.no_grad():
    for i in tqdm(range(100000)):   
        # create minibatch
        ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
        Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y    

        emb = C[Xb] # embed the characters into vectors
        embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
        # Linear layer 1
        hprebn = embcat @ W1 + b1 # hidden layer pre-activation
        # BatchNorm layer
        bnmeani = 1/n*hprebn.sum(0, keepdim=True)
        bndiff = hprebn - bnmeani
        bndiff2 = bndiff**2
        bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
        bnvar_inv = (bnvar + 1e-5)**-0.5
        bnraw = bndiff * bnvar_inv
        hpreact = bngain * bnraw + bnbias
        # Non-linearity
        h = torch.tanh(hpreact) # hidden layer
        # Linear layer 2
        logits = h @ W2 + b2 # output layer

        # cross entropy loss (same as F.cross_entropy(logits, Yb))
        logit_maxes = logits.max(1, keepdim=True).values
        norm_logits = logits - logit_maxes # subtract max for numerical stability bc we exponentiate in next step
        counts = norm_logits.exp()
        counts_sum = counts.sum(1, keepdims=True)
        counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
        probs = counts * counts_sum_inv
        logprobs = probs.log()

        loss = -logprobs[range(n), Yb].mean()

        # PyTorch manual backward pass without using loss.backward()
        for p in parameters:
            p.grad = None
        # loss.backward()
        # CE loss
        dlogits = F.softmax(logits, 1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n

        # Linear layer 2
        dh = dlogits @ W2.T # just match the shapes, there's math behind it but this is easier.
        dW2 = h.T @ dlogits # just match the shapes, there's math behind it but this is easier.
        db2 = dlogits.sum(0) # just match the shapes, there's math behind it but this is easier.

        # Non-linearity
        dhpreact = (1 - h**2) * dh

        # BatchNorm layer
        dbnraw = bngain * dhpreact
        dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
        dbnbias = dhpreact.sum(0, keepdim=True)
        dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)* bnraw * (dhpreact * bnraw).sum(0))

        # Linear layer 1
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)

        # Embedding layer
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k,j]
                dC[ix] += demb[k,j]

        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
        # Update parametersµ
        lr = 0.1 if i < 10000 else 0.01
        for grad, p in zip(grads, parameters):
            p.data -= lr * grad
        
        # Print loss
        if i % 10000 == 0:
            print(loss.item())
loss

100%|██████████| 100000/100000 [01:12<00:00, 1384.11it/s]


tensor(2.0528)

In [ ]:
# Check whether manually computed gradients match pytorch loss.backward gradients. You need to enable loss.backward() to use this fn without errors.
# for p,g in zip(parameters, grads):
#     cmp(str(tuple(p.shape)), g, p)

In [362]:
# Using batch norm during inference of single examples requires you to compute the mean and std of the training set and add that in as a hardcoded parameter
with torch.no_grad():
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    bnmean = hpreact.mean(0, keepdim=True)
    print(bnmean)
    bnstd = hpreact.std(0, keepdim=True)
    print(bnstd)

tensor([[-0.2738, -0.4173, -0.8774,  0.4045,  0.4551, -0.4792, -0.8698,  0.2897,
         -0.7849, -0.2291,  0.6210,  0.3900, -0.9105, -0.2744, -0.5740,  1.2438,
         -0.1958, -1.4369,  0.2378,  0.7349, -1.7816, -1.1060, -0.9264, -1.2399,
          0.6902, -0.0183, -0.3387,  1.2476, -0.4679,  0.1790,  0.2740, -0.1940,
          0.7042, -1.0832, -0.4859,  0.9816, -1.6462, -1.6222,  1.1056, -0.1111,
          0.3048,  0.4474,  0.4859, -1.1574,  1.8350,  1.0083,  0.3148, -0.2816,
          0.6530,  1.7979,  0.1401,  0.6169,  2.6471,  0.6642,  1.0196, -1.7593,
         -0.2628, -1.7576, -0.8058, -0.5995, -0.6516,  0.8525, -0.3123,  1.0368]])
tensor([[1.6294, 1.7009, 1.7275, 1.3175, 1.4074, 1.6327, 1.9597, 2.5524, 1.8503,
         1.6403, 1.6308, 1.6617, 1.8972, 1.5281, 1.4866, 2.0139, 1.9242, 2.5459,
         1.5011, 1.7737, 2.1074, 1.3808, 1.5716, 1.9338, 1.3493, 1.7105, 1.5049,
         1.4877, 1.9069, 1.9209, 1.7974, 1.8449, 1.5306, 1.4137, 1.5127, 2.2101,
         2.5424, 1.8396, 1

In [363]:
# Evaluation
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'dev': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) / bnstd + bnbias # batch norm

    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(f'{split} loss: {loss.item():.4f}')
split_loss('train')
split_loss('dev')
split_loss('test')

train loss: 2.1993
dev loss: 2.2077
test loss: 2.2082
